In [16]:
import os
import json
import pandas as pd
from openai import OpenAI
from dotenv import load_dotenv

In [50]:
# Read the CSV file into a Pandas DataFrame
# print(os.getcwd())
calendar_data = pd.read_csv("../data/calendar_data.csv")
print(len(calendar_data))

20


1. Loading the dataset and json
2. Prompt the dataset column and json question to gpt-4 to generate the python function
3. pass the generated funtion and dataframe into the test function
4. save the result into json-ansewer

In [37]:


with open("../data/question.json") as json_file:
    json_data = json.load(json_file)
    print(json_data[0])

{'question': 'How many meetings do I have scheduled in total?', 'answer': ''}


In [38]:
print(json_data[0]['question'])
print(len(json_data))

How many meetings do I have scheduled in total?
8


In [52]:
text = f"""
{json_data[0]['question']}
"""
prompt = f"""You are provided a DataFrame ```calendar_data(ID, status, summary, start, end , duration, attendees)```. This dataframe includes all you calenders.
Your task is generate python function to query this dataframe and answer the question. Output a python function object that function name is query.

The input have following columns:
- ID: meeting ID;
- status: meeting status, including the following status: cancelled, confirmed, tentative;
- summary: meeting or event topic;
- start: the start date of meeting;
- end: the start date of meeting;
- duration: meeting duration (second);
- attendees: people who attend the meeting delimited by "\n" with 1 sentence.

Output using the folowing format:
code language: python
function name: query():
input: calendar_data
output is a python function without any prefix instructionn and explaination.

For example, the output only have the following format:
def query(calendar_data):
    return calendar_data[0]

Question to be resolved: {text} 
"""

In [53]:
print(prompt)

 You are provided a DataFrame ```calendar_data(ID, status, summary, start, end , duration, attendees)```. This dataframe includes all you calenders.
Your task is generate python function to query this dataframe and answer the question. Output a python function object that function name is query.

The input have following columns:
- ID: meeting ID;
- status: meeting status, including the following status: cancelled, confirmed, tentative;
- summary: meeting or event topic;
- start: the start date of meeting;
- end: the start date of meeting;
- duration: meeting duration (second);
- attendees: people who attend the meeting delimited by "
" with 1 sentence.

Output using the folowing format:
code language: python
function name: query():
input: calendar_data
output is a python function without any prefix instructionn and explaination.

For example, the output only have the following format:
def query(calendar_data):
    return calendar_data[0]

Question to be resolved: 
How many meetings do

In [54]:

MODEL = "gpt-4"
# MODEL = "gpt-3.5-turbo"

client = OpenAI(
    # This is the default and can be omitted
    api_key=os.environ.get("OPENAI_API_KEY"),
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": prompt,
        }
    ],
    model=MODEL,
)

# verify the output
generated_text = chat_completion.choices[0].message.content
print(generated_text)

def query(calendar_data):
    return len(calendar_data)


In [55]:
fp = open('generated_code.py', 'w')
fp.write(generated_text)
fp.close()

In [68]:
from generated_code import query

answer = "Error"
 
try:
  print(json_data[0]['question']) 
  answer = query(calendar_data)
  print(query(calendar_data))
except:
  print("An exception occurred")

How many meetings do I have scheduled in total?
20


Save the result into the .json file